# Build and run models

## Ways of defining models

In essence, most population pharmacokinetic models are ordinary differential equations (ODEs) to describe the distribution of drug molecules. Hence, there are three approaches to writing a PopPK model in the Maspectra `masmod` library:

1. **Use the built-in PK library**.
2. **Closed-form equations:** Express the structural model with the closed-form solution for the ODE equations.
3. **ODE models:** Define user-customized compartments and the corresponding ODEs.

### Use the built-in PK library

Most commonly-used PK compartment models are implemented in Maspectra model library. They are fast and easy-to-use.

The builtin compartment models are essentially solved ODEs. The compartments in the built-in models are pre-defined and cannot be modified.

For example,

In [8]:
from mas.model import *

class DemoOralOneCmtModelWithBuiltInLibrary(pk.EvOneCmtLinear):
    def __init__(self) -> None:
        super().__init__()

        self.tv_cl = theta(0.2)
        self.tv_v = theta(9)
        self.tv_ka = theta(1.3)

        self.eta_cl = omega(0.1)
        self.eta_v = omega(0.1)
        self.eta_ka = omega(0.1)

        self.eps_prop = sigma(0.01)
        self.eps_add = sigma(0.15)

    def pred(self) -> Expression:
        cl = self.tv_cl * exp(self.eta_cl)
        v = self.tv_v * exp(self.eta_v)
        ka = self.tv_ka * exp(self.eta_ka)

        # to use the builtin PK library with the specified parameterization
        computed = self.trans2(cl=cl, v=v, ka=ka)

        ipred = computed.F
        y = ipred * (1 + self.eps_prop) + self.eps_add

        return y

### Closed-form equations

If the closed form equations of the ODEs is known, we may directly use the equations to write the models. For example

In [9]:
from mas.model import *

class DemoOralOneCmtModelWithClosedForm(Module):
    def __init__(self) -> None:
        super().__init__()

        self.tv_cl = theta(0.2)
        self.tv_v = theta(9)
        self.tv_ka = theta(1.3)

        self.eta_cl = omega(0.1)
        self.eta_v = omega(0.1)
        self.eta_ka = omega(0.1)

        self.eps_prop = sigma(0.01)
        self.eps_add = sigma(0.15)

        # retrieve the dose amounts from the ModelData 
        self.dose = column('DOSE')

    def pred(self) -> Expression:
        cl = self.tv_cl * exp(self.eta_cl)
        v = self.tv_v * exp(self.eta_v)
        ka = self.tv_ka * exp(self.eta_ka)

        k = cl / v

        # closed form equation for the solved ODEs
        conc = self.dose / v * ka / (ka - k) * (exp(-k * self.t) - exp(-ka * self.t))

        ipred = conc
        y = ipred * (1 + self.eps_prop) + self.eps_add

        return y

Note that here dose is input as a data column, so we must make sure there is a column in each row to indicate the dose amount.

### ODE models

ODE models provide more flexibility, especially when the structure model is complex/mechanism-related.

First, the compartments must be defined in the `__init__` method. We can specify which compartment is the default for dosing and which is for observation.

The ODEs are defined in the `pred` method.

The following example demonstrates an oral one-compartment model identical to the models defined above.

In [10]:
from mas.model import *

class DemoOralOneCmtModelWithODE(Module):
    def __init__(self) -> None:
        super().__init__()

        self.tv_cl = theta(0.2)
        self.tv_v = theta(9)
        self.tv_ka = theta(1.3)

        self.eta_cl = omega(0.1)
        self.eta_v = omega(0.1)
        self.eta_ka = omega(0.1)

        self.eps_prop = sigma(0.01)
        self.eps_add = sigma(0.15)

        # definition of compartments
        self.cmt_depot = compartment(default_dose=True)
        self.cmt_central = compartment(default_obs=True)

    def pred(self) -> Expression:
        cl = self.tv_cl * exp(self.eta_cl)
        v = self.tv_v * exp(self.eta_v)
        ka = self.tv_ka * exp(self.eta_ka)

        k = cl / v

        # definition of ordinary differential equations 
        self.cmt_depot.dAdt = -ka * self.cmt_depot.A
        self.cmt_central.dAdt = ka * self.cmt_depot.A - k * self.cmt_central.A

        ipred = self.cmt_central.A / v
        y = ipred * (1 + self.eps_prop) + self.eps_add

        return y

## Model estimation

Maspectra supports varies of model estimation algorithms:

- Laplacian methods: FO/FOCEi/Laplace
- EM methods: SAEM

## Model diagnostics

Model estimation results can be evaluated via diagnostic plots:

- Goodness-of-fit plots: this can be produced via `FitResult.plot_goodness_of_fit()`
- Individual fit plots: this can be produced via `FitResult.plot_individual_fit()`

We also provide commonly-used diagnostic tools such as Visual Predictive Checks (VPC) and Boostrap.